In [8]:
print(d)

{'train': [[3.6140742, 7.227595, 6.70115, 3.6506257, 6.622177, 3.5594428, 5.5601854, 4.5101857, 3.3610966, 9.822557, 6.1879897, 7.4154177, 3.217002], [3.5472631, 7.108092, 6.5936656, 3.5849123, 6.519177, 3.498232, 5.4680862, 4.434257, 3.303298, 9.674823, 6.095251, 7.3001394, 3.158329], [3.4862378, 6.9968557, 6.4919667, 3.5228007, 6.420512, 3.4389234, 5.378127, 4.3604865, 3.2469144, 9.530659, 6.004282, 7.186862, 3.1008427], [3.4265602, 6.888028, 6.392318, 3.462064, 6.3238683, 3.38082, 5.289957, 4.2882223, 3.191661, 9.389341, 5.9149823, 7.0757785, 3.0445414], [3.3680334, 6.781307, 6.2945023, 3.402525, 6.2290397, 3.323862, 5.2035356, 4.2173944, 3.1375017, 9.250699, 5.8273, 6.96684, 2.9893854], [3.310612, 6.676588, 6.198433, 3.3441286, 6.135934, 3.2680173, 5.1187944, 4.1479607, 3.0844135, 9.114625, 5.7411876, 6.8599553, 2.9353518], [3.2542937, 6.57382, 6.104082, 3.286867, 6.0445, 3.2132654, 5.035672, 4.079875, 3.0323608, 8.981022, 5.6565876, 6.7550373, 2.882391], [3.199048, 6.4729385, 6.01

In [36]:
screenshot.keys()

dict_keys(['W', 'alpha', 'eps', 'y_hat'])

### Check out Pytorch model

In [1]:
import torch
import torch.nn as nn
import cellbox
import os
import numpy as np
import pandas as pd
import shutil
import argparse
import json
import glob
import time
from cellbox.utils import TimeLogger

import tensorflow.compat.v1 as tf
from tensorflow.compat.v1.errors import OutOfRangeError
tf.disable_v2_behavior()

2023-07-12 14:34:29.044140: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-12 14:34:36.527694: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-12 14:34:58.904985: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/lsf10/10.1/linux3.10-glibc2.17-x86_64/lib:/data/weirauchlab/opt/lib:/da

Instructions for updating:
non-resource variables are not supported in the long term
   _____     _ _ ____              
  / ____|   | | |  _ \             
 | |     ___| | | |_) | _____  __  
 | |    / _ \ | |  _ < / _ \ \/ /  
 | |___|  __/ | | |_) | (_) >  <   
  \_____\___|_|_|____/ \___/_/\_\  
Running CellBox scripts developed in Sander lab
Maintained by Bo Yuan, Judy Shen, and Augustin Luna; contributions by Daniel Ritter

        version 0.3.2
        -- Feb 10, 2023 --
        * Modify CellBox to support TF2     
        
Tutorials and documentations are available at https://github.com/sanderlab/CellBox
If you want to discuss the usage or to report a bug, please use the 'Issues' function at GitHub.
If you find CellBox useful for your research, please consider citing the corresponding publication.
For more information, please email us at boyuan@g.harvard.edu and c_shen@g.harvard.edu, augustin_luna@hms.harvard.edu
 ----------------------------------------------------------------

In [2]:
def set_seed(in_seed):
    int_seed = int(in_seed)
    tf.compat.v1.set_random_seed(int_seed)
    np.random.seed(int_seed)


def prepare_workdir(in_cfg):
    # Read Data
    in_cfg.root_dir = os.getcwd()
    in_cfg.node_index = pd.read_csv(in_cfg.node_index_file, header=None, names=None) \
        if hasattr(in_cfg, 'node_index_file') else pd.DataFrame(np.arange(in_cfg.n_x))

    # Create Output Folder
    experiment_path = 'results/{}_{}'.format(in_cfg.experiment_id, md5)
    try:
        os.makedirs(experiment_path)
    except Exception:
        pass
    out_cfg = vars(in_cfg)
    out_cfg = {key: out_cfg[key] for key in out_cfg if type(out_cfg[key]) is not pd.DataFrame}
    os.chdir(experiment_path)
    json.dump(out_cfg, open('config.json', 'w'), indent=4)

    if "leave one out" in in_cfg.experiment_type:
        try:
            in_cfg.model_prefix = '{}_{}'.format(in_cfg.model_prefix, in_cfg.drug_index)
        except Exception('Drug index not specified') as e:
            raise e

    in_cfg.working_index = in_cfg.model_prefix + "_" + str(working_index).zfill(3)

    try:
        shutil.rmtree(in_cfg.working_index)
    except Exception:
        pass
    os.makedirs(in_cfg.working_index)
    os.chdir(in_cfg.working_index)

    with open("record_eval.csv", 'w') as f:
        f.write("epoch,iter,train_loss,valid_loss,train_mse,valid_mse,test_mse,time_elapsed\n")

    print('Working directory is ready at {}.'.format(experiment_path))
    return 0

experiment_config_path = "/users/ngun7t/Documents/cellbox-jun-6/configs_dev/Example.random_partition.CellBox.json"
working_index = 0
stage = {
    "nT": 100,
    "sub_stages":[
        {"lr_val": 0.1,"l1lambda": 0.01, "n_iter_patience":1000},
        {"lr_val": 0.01,"l1lambda": 0.01},
        {"lr_val": 0.01,"l1lambda": 0.0001},
        {"lr_val": 0.001,"l1lambda": 0.00001}
    ]}

cfg = cellbox.config.Config(experiment_config_path)
cfg.ckpt_path_full = os.path.join('./', cfg.ckpt_name)
md5 = cellbox.utils.md5(cfg)
cfg.drug_index = 5         # Change this for testing purposes
cfg.seed = working_index + cfg.seed if hasattr(cfg, "seed") else working_index + 1000
set_seed(cfg.seed)
print(vars(cfg))

prepare_workdir(cfg)
logger = cellbox.utils.TimeLogger(time_logger_step=1, hierachy=3)
args = cfg
for i, stage in enumerate(cfg.stages):
    set_seed(cfg.seed)
    cfg = cellbox.dataset_torch.factory(cfg)
    args.sub_stages = stage['sub_stages']
    args.n_T = stage['nT']
    model = cellbox.model_torch.factory(args)
    if i == 0: break

{'experiment_id': 'Example_RP', 'model_prefix': 'seed', 'ckpt_name': 'model11.ckpt', 'export_verbose': 3, 'experiment_type': 'random partition', 'sparse_data': False, 'batchsize': 4, 'trainset_ratio': 0.7, 'validset_ratio': 0.8, 'n_batches_eval': None, 'add_noise_level': 0, 'dT': 0.1, 'ode_solver': 'heun', 'envelope_form': 'tanh', 'envelope': 0, 'pert_form': 'by u', 'ode_degree': 1, 'ode_last_steps': 2, 'n_iter_buffer': 50, 'n_iter_patience': 100, 'weight_loss': 'None', 'l1lambda': 0.0001, 'l2lambda': 0.0001, 'model': 'CellBox', 'pert_file': '/users/ngun7t/Documents/cellbox-jun-6/data/pert.csv', 'expr_file': '/users/ngun7t/Documents/cellbox-jun-6/data/expr.csv', 'node_index_file': '/users/ngun7t/Documents/cellbox-jun-6/data/node_Index.csv', 'n_protein_nodes': 82, 'n_activity_nodes': 87, 'n_x': 99, 'envelop_form': 'tanh', 'envelop': 0, 'n_epoch': 100, 'n_iter': 100, 'stages': [{'nT': 100, 'sub_stages': [{'lr_val': 0.1, 'l1lambda': 0.01, 'n_iter_patience': 1000}, {'lr_val': 0.01, 'l1lamb

In [31]:
def train_substage(model, lr_val, l1_lambda, l2_lambda, n_epoch, n_iter, n_iter_buffer, n_iter_patience, args):

    stages = glob.glob("*best*.csv")
    try:
        substage_i = 1 + max([int(stage[0]) for stage in stages])
    except Exception:
        substage_i = 1

    best_params = Screenshot(args, n_iter_buffer)

    n_unchanged = 0
    idx_iter = 0
    #for key in args.feed_dicts:
    #    args.feed_dicts[key].update({
    #        model.lr: lr_val,
    #        model.l1_lambda: l1_lambda,
    #        model.l2_lambda: l2_lambda
    #    })
    args.logger.log("--------- lr: {}\tl1: {}\tl2: {}\t".format(lr_val, l1_lambda, l2_lambda))
    args.l1_lambda = l1_lambda
    args.l2_lambda = l2_lambda
    args.lr = lr_val
    args.optimizer = cellbox.utils_torch.optimize(
        model.parameters(),
        lr=args.lr
    )

    #sess.run(model.iter_monitor.initializer, feed_dict=args.feed_dicts['valid_set'])
    for idx_epoch in range(n_epoch):

        if idx_iter > n_iter or n_unchanged > n_iter_patience:
            break

        for i, train_minibatch in enumerate(args.iter_train):
            # Each train_minibatch has shape of (batch_size, num_features)
            x_train, y_train = train_minibatch

            if idx_iter > n_iter or n_unchanged > n_iter_patience:
                break

            # Do one forward pass
            t0 = time.perf_counter()
            model.train()
            model.zero_grad()
            args.optimizer.zero_grad()
            if args.pert_form == "by u":
                prediction = model(torch.zeros((args.n_x, 1), dtype=torch.float32).to(args.device), x_train.to(args.device))
            elif args.pert_form == "fix x":
                prediction = model(x_train.T.to(args.device), x_train.to(args.device))
            convergence_metric, yhat = prediction

            for param in model.named_parameters():
                if param[0] == "params.W":
                    param_mat = param[1]
                    break
            #loss_train_i, loss_train_mse_i = args.loss_fn(y_train.to(args.device), yhat, model.state_dict()["params.W"], l1=l1_lambda, l2=l2_lambda)
            loss_train_i, loss_train_mse_i = args.loss_fn(y_train.to(args.device), yhat, param_mat, l1=l1_lambda, l2=l2_lambda, weight=y_train.to(args.device))
            # Check the gradients of the model
            #params = model.parameters()
            #for item in params:
            #    item.register_hook(lambda grad: print(grad))
                #print(f"Grad of before backward: {item.grad}")

            loss_train_i.backward()
            #params = model.state_dict()
            #for item in params:
            #    print(f"Grad of {item} after backward: {params[item].grad}")
            args.optimizer.step()

            # Record training
            with torch.no_grad():
                # Very questionable for validation
                model.eval()
                valid_minibatch = iter(args.iter_monitor)
                x_valid, y_valid = next(valid_minibatch)
                if args.pert_form == "by u":
                    prediction = model(
                        torch.zeros((args.n_x, 1), dtype=torch.float32).to(args.device), 
                        x_valid.to(args.device)
                    )
                elif args.pert_form == "fix x":
                    prediction = model(
                        x_valid.T.to(args.device),
                        x_valid.to(args.device)
                    )
                    
                convergence_metric, yhat = prediction

                for param in model.named_parameters():
                    if param[0] == "params.W":
                        param_mat = param[1]
                        break
                #loss_valid_i, loss_valid_mse_i = args.loss_fn(y_valid.to(args.device), yhat, model.state_dict()["params.W"], l1=l1_lambda, l2=l2_lambda)
                loss_valid_i, loss_valid_mse_i = args.loss_fn(y_valid.to(args.device), yhat, param_mat, l1=l1_lambda, l2=l2_lambda, weight=y_valid.to(args.device))

            # Record results to screenshot
            new_loss = best_params.avg_n_iters_loss(loss_valid_i)
            if args.export_verbose > 0:
                print(("Substage:{}\tEpoch:{}/{}\tIteration: {}/{}" + "\tloss (train):{:1.6f}\tloss (buffer on valid):"
                       "{:1.6f}" + "\tbest:{:1.6f}\tTolerance: {}/{}").format(substage_i, idx_epoch, n_epoch, idx_iter,
                                                                              n_iter, loss_train_i, new_loss,
                                                                              best_params.loss_min, n_unchanged,
                                                                              n_iter_patience))
            
            append_record("record_eval.csv",
                          [idx_epoch, idx_iter, loss_train_i.item(), loss_valid_i.item(), loss_train_mse_i.item(),
                           loss_valid_mse_i.item(), None, time.perf_counter() - t0])

            # Early stopping
            idx_iter += 1
            if new_loss < best_params.loss_min:
                n_unchanged = 0
                best_params.screenshot(model, substage_i, args=args,
                                       node_index=args.dataset['node_index'], loss_min=new_loss)
            else:
                n_unchanged += 1

        for k, v in best_params.items():
            assert type(v) == pd.DataFrame, print(k)

    # Evaluation on valid set
    t0 = time.perf_counter()
    loss_valid_i = eval_model(
        args, args.iter_monitor, model, return_value="loss_full", n_batches_eval=args.n_batches_eval
    )
    loss_valid_mse_i = eval_model(
        args, args.iter_monitor, model, return_value="loss_mse", n_batches_eval=args.n_batches_eval
    )
    append_record("record_eval.csv", [-1, None, None, loss_valid_i, None, loss_valid_mse_i, None, time.perf_counter() - t0])

    # Evaluation on test set
    t0 = time.perf_counter()
    loss_test_mse = eval_model(
        args, args.iter_eval, model, return_value="loss_mse", n_batches_eval=args.n_batches_eval
    )
    append_record("record_eval.csv", [-1, None, None, None, None, None, loss_test_mse, time.perf_counter() - t0])

    # Save results
    best_params.save()
    args.logger.log("------------------ Substage {} finished!-------------------".format(substage_i))
    save_model(model, f"./model11.pth")


    #sess.run(model.iter_eval.initializer, feed_dict=args.feed_dicts['valid_set'])
    #loss_valid_i, loss_valid_mse_i = eval_model(sess, model.iter_eval, (model.eval_loss, model.eval_mse_loss),
    #                                            args.feed_dicts['valid_set'], n_batches_eval=args.n_batches_eval)
    #append_record("record_eval.csv", [-1, None, None, loss_valid_i, None, loss_valid_mse_i, None, time.perf_counter() - t0])
#
    ## Evaluation on test set
    #t0 = time.perf_counter()
    #sess.run(model.iter_eval.initializer, feed_dict=args.feed_dicts['test_set'])
    #loss_test_mse = eval_model(sess, model.iter_eval, model.eval_mse_loss,
    #                           args.feed_dicts['test_set'], n_batches_eval=args.n_batches_eval)
    #append_record("record_eval.csv", [-1, None, None, None, None, None, loss_test_mse, time.perf_counter() - t0])
#
    #best_params.save()
    #args.logger.log("------------------ Substage {} finished!-------------------".format(substage_i))
    #save_model(args.saver, sess, './' + args.ckpt_name)

    

    return best_params


def append_record(filename, contents):
    """define function for appending training record"""
    with open(filename, 'a') as f:
        for content in contents:
            f.write('{},'.format(content))
        f.write('\n')


def eval_model(args, eval_iter, model, return_value, return_avg=True, n_batches_eval=None):
    """ Simulate the model for prediction """

    with torch.no_grad():
        counter = 0
        eval_results = []
        for item in eval_iter:
            pert, expr = item
            if args.pert_form == "by u":
                prediction = model(
                    torch.zeros((args.n_x, 1), dtype=torch.float32).to(args.device), 
                    pert.to(args.device)
                )
            elif args.pert_form == "fix x":
                prediction = model(
                    pert.T.to(args.device), 
                    pert.to(args.device)
                )
            _, yhat = prediction
            if return_value == "prediction":
                eval_results.append(yhat.detach().cpu().numpy())
            elif return_value == "loss_full":
                loss_full, _ = args.loss_fn(expr.to(args.device), yhat, model.state_dict()["params.W"], l1=args.l1_lambda, l2=args.l2_lambda)
                eval_results.append(loss_full.detach().cpu().numpy())
            elif return_value == "loss_mse":
                _, loss_mse = args.loss_fn(expr.to(args.device), yhat, model.state_dict()["params.W"], l1=args.l1_lambda, l2=args.l2_lambda)
                eval_results.append(loss_mse.detach().cpu().numpy())
            counter += 1
            if n_batches_eval is not None and counter > n_batches_eval:
                break

        if return_avg:
            return np.mean(np.array(eval_results), axis=0)
        return np.vstack(eval_results)
    

def save_model(model, save_dir):
    """ Save the model """
    torch.save(model.state_dict(), save_dir)


def train_model(model, args):
    """Train the model"""
    args.logger = TimeLogger(time_logger_step=1, hierachy=2)
    model = model[0].to(args.device)

    # Check if all variables in scope
    # TODO: put variables under appropriate scopes
    #try:
    #    args.saver.restore(sess, './' + args.ckpt_name)
    #    print('Load existing model at {}...'.format(args.ckpt_name))
    #except Exception:
    #    print('Create new model at {}...'.format(args.ckpt_name))

    # Training
    for substage in args.sub_stages:
        n_iter_buffer = substage['n_iter_buffer'] if 'n_iter_buffer' in substage else args.n_iter_buffer
        n_iter = substage['n_iter'] if 'n_iter' in substage else args.n_iter
        n_iter_patience = substage['n_iter_patience'] if 'n_iter_patience' in substage else args.n_iter_patience
        n_epoch = substage['n_epoch'] if 'n_epoch' in substage else args.n_epoch
        l1 = substage['l1lambda'] if 'l1lambda' in substage else args.l1lambda if hasattr(args, 'l1lambda') else 0
        l2 = substage['l2lambda'] if 'l2lambda' in substage else args.l2lambda if hasattr(args, 'l2lambda') else 0
        screenshot = train_substage(model, substage['lr_val'], l1_lambda=l1, l2_lambda=l2, n_epoch=n_epoch,
                       n_iter=n_iter, n_iter_buffer=n_iter_buffer, n_iter_patience=n_iter_patience, args=args)

    return screenshot
        

class Screenshot(dict):
    """summarize the model"""
    def __init__(self, args, n_iter_buffer):
        # initialize loss_min
        super().__init__()
        self.loss_min = 1000
        # initialize tuning_metric
        self.saved_losses = [self.loss_min]
        self.n_iter_buffer = n_iter_buffer
        # initialize verbose
        self.summary = {}
        self.summary = {}
        self.substage_i = []
        self.export_verbose = args.export_verbose
        self.args = args

    def avg_n_iters_loss(self, new_loss):
        """average the last few losses"""
        self.saved_losses = self.saved_losses + [new_loss]
        self.saved_losses = self.saved_losses[-self.n_iter_buffer:]
        return sum(self.saved_losses) / len(self.saved_losses)

    def screenshot(self, model, substage_i, node_index, loss_min, args):
        """evaluate models"""
        self.substage_i = substage_i
        self.loss_min = loss_min

        # Save the variable weights associated with each of the conditions in a csv file
        if self.export_verbose > 0:
            #layer = model.W
            params = model.state_dict()
            new_params = {}
            for item in params:
                try:
                    new_params[item] = pd.DataFrame(params[item].detach().numpy(), index=node_index[0])
                except Exception:
                    new_params[item] = pd.DataFrame(params[item].detach().numpy())
            self.update(new_params)

        if self.export_verbose > 1 or self.export_verbose == -1:  # no params but y_hat
            y_hat = eval_model(args, args.iter_eval, model, return_value="prediction", return_avg=False)
            y_hat = pd.DataFrame(y_hat, columns=node_index[0])
            self.update({'y_hat': y_hat})

        if self.export_verbose > 2:
            try:
                # Run summary on train set
                converge_train_mat, converge_eval_mat, converge_test_mat = [], [], []
                for item in args.iter_train:
                    pert, _ = item
                    if args.pert_form == "by u":
                        prediction = model(
                            torch.zeros((args.n_x, 1), dtype=torch.float32).to(args.device), 
                            pert.to(args.device)
                        )
                    elif args.pert_form == "fix x":
                        prediction = model(
                            pert.T.to(args.device), 
                            pert.to(args.device)
                        )
                    # Shape (length, batch_size)
                    convergence_metric_train, _ = prediction
                    converge_train_mat.append(convergence_metric_train.detach().numpy())

                # Run summary on eval set
                for item in args.iter_monitor:
                    pert, _ = item
                    if args.pert_form == "by u":
                        prediction = model(
                            torch.zeros((args.n_x, 1), dtype=torch.float32).to(args.device), 
                            pert.to(args.device)
                        )
                    elif args.pert_form == "fix x":
                        prediction = model(
                            pert.T.to(args.device), 
                            pert.to(args.device)
                        )
                    # Shape (length, batch_size)
                    convergence_metric_eval, _ = prediction
                    converge_eval_mat.append(convergence_metric_eval.detach().numpy())
                
                # Run summary on test set
                for item in args.iter_eval:
                    pert, _ = item
                    if args.pert_form == "by u":
                        prediction = model(
                            torch.zeros((args.n_x, 1), dtype=torch.float32).to(args.device), 
                            pert.to(args.device)
                        )
                    elif args.pert_form == "fix x":
                        prediction = model(
                            pert.T.to(args.device), 
                            pert.to(args.device)
                        )
                    # Shape (length, batch_size)
                    convergence_metric_test, _ = prediction
                    converge_test_mat.append(convergence_metric_test.detach().numpy())

                # Concatenate the results:
                converge_train_mat = np.concatenate(converge_train_mat, axis=1)
                converge_test_mat = np.concatenate(converge_test_mat, axis=1)
                converge_eval_mat = np.concatenate(converge_eval_mat, axis=1)
                
                # Summarize performance
                cols = [node_index.values + '_mean', node_index.values + '_sd', node_index.values + '_dxdt']
                cols = np.squeeze(np.concatenate(cols)).tolist()
                summary_train = pd.DataFrame(converge_train_mat.T, columns=cols)
                summary_test = pd.DataFrame(converge_test_mat.T, columns=cols)
                summary_valid = pd.DataFrame(converge_eval_mat.T, columns=cols)
                self.update(
                    {'summary_train': summary_train, 'summary_test': summary_test, 'summary_valid': summary_valid}
                )

            except Exception as e:
                print(e)

    def save(self):
        """save model parameters"""
        for file in glob.glob(str(self.substage_i) + "_best.*.csv"):
            os.remove(file)
        for key in self:
            self[key].to_csv("{}_best.{}.loss.{}.csv".format(self.substage_i, key, self.loss_min))
        


In [32]:
args.device = torch.device("cpu")

In [33]:
screenshot = train_model(model, args)

########   --------- lr: 0.1	l1: 0.01	l2: 0.0001	   --time elapsed: 0.00
Substage:1	Epoch:0/100	Iteration: 0/100	loss (train):65.206741	loss (buffer on valid):533.057373	best:1000.000000	Tolerance: 0/1000


Substage:1	Epoch:0/100	Iteration: 1/100	loss (train):65.931412	loss (buffer on valid):377.345947	best:533.057373	Tolerance: 0/1000
Substage:1	Epoch:0/100	Iteration: 2/100	loss (train):66.116508	loss (buffer on valid):299.499664	best:377.345947	Tolerance: 0/1000
Substage:1	Epoch:0/100	Iteration: 3/100	loss (train):65.780098	loss (buffer on valid):252.872162	best:299.499664	Tolerance: 0/1000
Substage:1	Epoch:0/100	Iteration: 4/100	loss (train):65.498451	loss (buffer on valid):221.606552	best:252.872162	Tolerance: 0/1000
Substage:1	Epoch:0/100	Iteration: 5/100	loss (train):65.409477	loss (buffer on valid):199.208847	best:221.606552	Tolerance: 0/1000
Substage:1	Epoch:0/100	Iteration: 6/100	loss (train):65.636185	loss (buffer on valid):182.340973	best:199.208847	Tolerance: 0/1000
Substage:1	Epoch:0/100	Iteration: 7/100	loss (train):65.217102	loss (buffer on valid):169.192276	best:182.340973	Tolerance: 0/1000
Substage:1	Epoch:0/100	Iteration: 8/100	loss (train):63.767570	loss (buffer on vali

KeyboardInterrupt: 

In [4]:
actual_model = model[0]
actual_model.load_state_dict(torch.load("/users/ngun7t/Documents/cellbox-jun-6/results/Example_RP_369eb781d89dcf91404cf96a8fefb773/seed_000/model11.pth"))
actual_model.eval()

CellBox(
  (params): ParameterDict(
      (W): Parameter containing: [torch.FloatTensor of size 99x99]
      (alpha): Parameter containing: [torch.FloatTensor of size 99x1]
      (eps): Parameter containing: [torch.FloatTensor of size 99x1]
  )
)

In [9]:
for param in actual_model.parameters():
    print(param.requires_grad)


True
True
True


In [25]:
for thing in actual_model.named_parameters():
    print(thing[1].requires_grad)

True
True
True


In [17]:
params = actual_model.state_dict()
print(type(params["params.W"]))
print(params["params.W"].requires_grad)


<class 'torch.Tensor'>
False
params.W
params.alpha
params.eps


In [ ]:
a = torch.nn.Parameter()

In [18]:
n_epoch = 10
for idx_epoch in range(n_epoch):

    for i, train_minibatch in enumerate(args.iter_train):
        # Each train_minibatch has shape of (batch_size, num_features)
        x_train, y_train = train_minibatch

        # Do one forward pass
        t0 = time.perf_counter()
        actual_model.train()
        args.optimizer.zero_grad()
        if args.pert_form == "by u":
            prediction = actual_model(torch.zeros((args.n_x, 1), dtype=torch.float32).to(args.device), x_train.to(args.device))
        elif args.pert_form == "fix x":
            prediction = actual_model(x_train.T.to(args.device), x_train.to(args.device))
        convergence_metric, yhat = prediction
        loss_train_i, loss_train_mse_i = args.loss_fn(y_train.to(args.device), yhat, actual_model.state_dict()["params.W"], l1=args.l1_lambda, l2=args.l2_lambda)
        loss_train_i.backward()
        args.optimizer.step()

        print(f"loss_train_i: {loss_train_i.item()}")

loss_train_i: 81.7782974243164
loss_train_i: 82.3116455078125
loss_train_i: 82.89201354980469
loss_train_i: 83.25794982910156
loss_train_i: 83.6027603149414
loss_train_i: 83.91826629638672
loss_train_i: 84.33826446533203
loss_train_i: 84.68413543701172
loss_train_i: 84.81233215332031
loss_train_i: 85.213623046875
loss_train_i: 85.30145263671875
loss_train_i: 85.51406860351562
loss_train_i: 85.80474853515625
loss_train_i: 86.03447723388672
loss_train_i: 85.9633560180664
loss_train_i: 86.2061538696289
loss_train_i: 86.38774108886719
loss_train_i: 86.35466766357422
loss_train_i: 86.42328643798828
loss_train_i: 86.5203857421875
loss_train_i: 86.67794036865234
loss_train_i: 86.6491470336914
loss_train_i: 86.88619232177734
loss_train_i: 86.7999038696289
loss_train_i: 86.7457504272461
loss_train_i: 86.94552612304688
loss_train_i: 86.96121215820312
loss_train_i: 86.96076202392578
loss_train_i: 86.84939575195312
loss_train_i: 87.06599426269531
loss_train_i: 87.01042175292969
loss_train_i: 87.04

In [14]:
screenshot.keys()

dict_keys(['params.W', 'params.alpha', 'params.eps', 'y_hat', 'summary_train', 'summary_test', 'summary_valid'])

In [18]:
type(screenshot["summary_train"])

pandas.core.frame.DataFrame

In [17]:
torch.device("cuda")

device(type='cuda')